In [1]:
from make_predictions import *

FILE_PATH = '/home/sam/Hhd/twitter_sentiment/'
# FILE_PATH = '/home/sam/Data/twitter_sentiment/'

Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


### AirCanada

In [2]:
AC = GenerateData(FILE_PATH, 'AirCanada.csv', max_len_train=19)
AC.prepare_data(wv_size=600)
AC.get_result(n_preview=10, n_top = 20, name='AC_result',verbose=False)

Loading csv: AirCanada.csv ...
Note: pre-process changes the dataframe inplace.
Loading existing model tweets600.model.bin ...
Done building.
max sentence length is:  17
npy already exists, loading ...
Done loading npy file.
npy already exists.
>>>Done preparing data.<<<

prediction
Done converting numeric to categorical, this changes df.
===Positive===
[('great', 8), ('thanks', 7), ('thank', 4), ('home', 3), ('today', 3), ('help', 2), ('go', 2), ('lounge', 2), ('jetxd', 2), ('lhr', 2), ('made', 2), ('yyz', 2), ('bag', 2), ('pbi', 2), ('bad', 2), ('working', 2), ('visit', 2), ('service', 2), ('flying', 2), ('time', 2)]
===Neutral===
[('flight', 15), ('support', 11), ('travel', 10), ('trip', 9), ('big', 9), ('vancouver', 8), ('change', 7), ('fly', 7), ('seat', 7), ('via', 6)]
===Negative===
[('flight', 35), ('time', 12), ('hours', 11), ('found', 10), ('worst', 9), ('didnt', 9), ('actually', 9), ('board', 9), ('check', 9), ('service', 8), ('hrs', 8), ('yvr', 7), ('never', 7), ('already',

In [4]:
AC.check(word='worst', sentiment=3, n_view=10)

@AirCanada you are the worst. I paid 4 a flight with a guaranteed seat. but of course u over booked. Now I have a... https://t.co/mxxQfETfH6
@AirCanada you are the worst. I paid 4 a flight with a guaranteed seat. but of course u over booked. Now I have a delay of 6 hours #boycott
@AirCanada you are the worst. Each experience worse than the last.  Too bad we don't have a choice of air carriers.
@AirCanada is absolute worst airline. Rude staff, complete incompetence, left bags off flight - then yelled at us for asking after them
Just had the worst service experience ever on @AirCanada, I'll never flight again with you!
RT @mcjackson001: @AirCanada Absolute worst airline I have ever flown!!! NEVER flying them again. Take a cue from @SouthwestAir in the cust…
@AirCanada Absolute worst airline I have ever flown!!! NEVER flying them again. Take a cue from @SouthwestAir in the customer service dept.
RT @gattaca: Hey @AirCanada just got off AC973, onboard service was the worst I’ve ever experi

### AmericanAir

In [5]:
AA = GenerateData(FILE_PATH, 'Americanair.csv', max_len_train=19)
AA.prepare_data(wv_size=600)
AA.get_result(n_preview=10, n_top = 20, name='AC_result',verbose=False)

Loading csv: Americanair.csv ...
Note: pre-process changes the dataframe inplace.
Loading existing model tweets600.model.bin ...
Done building.
max sentence length is:  18
npy already exists, loading ...
Done loading npy file.
npy already exists.
>>>Done preparing data.<<<

prediction
Done converting numeric to categorical, this changes df.
===Positive===
[('great', 9), ('thanks', 8), ('xd', 7), ('flight', 4), ('thank', 3), ('big', 3), ('nigger', 3), ('much', 3), ('go', 2), ('mill', 2), ('lets', 2), ('friendly', 2), ('looking', 2), ('enough', 2), ('tour', 2), ('airline', 2), ('family', 2), ('ppl', 2), ('speak', 2), ('forward', 2)]
===Neutral===
[('please', 49), ('eduaubdedubu', 49), ('shortcut', 48), ('info', 48), ('visit', 48), ('watch', 11), ('flight', 9), ('airfare', 5), ('charter', 5), ('roundtrip', 5)]
===Negative===
[('flight', 20), ('hour', 10), ('flights', 10), ('plane', 9), ('im', 9), ('dont', 8), ('service', 8), ('people', 8), ('time', 8), ('hours', 7), ('worst', 7), ('get', 

In [6]:
AA.check(word='worst', sentiment=3, n_view=10)

Siobhan: "The worst, @AmericanAir. And no, everyone does not [travel w kids]. In fact, most people do not have a clue how hard it is."
Hey @Delta you're literally the worst. You're worse than @AmericanAir and that's pretty awful.
Natalie: "@AmericanAir is the WORST when traveling with kids."
@MissSaintXXX @AmericanAir Flying is the worst!!  Give me trains or car roadies all the time!!
No one ever fly @AmericanAir the worst experience of my life I was supposed to be home over 24 hours ago
@AmericanAir flight 1994 leaving PHL for ORD just had he longest and worst executed boarding process I've ever seen.. no hyperbole. #Flying
@AmericanAir is the worst airline in the entire world.  It should not have the word America in it.  This is why I fly @Delta. Figure it out
